# Business Understanding

The CRISP-DM methodolgy is used on that project.

*   Business Understanding
*   Data Understanding
*   Data Preparetion
*   Modelling
*   Evuluation
*   Deployment

**My Notes**
- Passengers in a group are often family but not always, >=_02 in the passenger ID it means that it is a group travelling.
- Cabin is split up into deck/num/side
- RoomService, FoodCourt, ShoppingMall, Spa, VRDeck - monetary values in terms of amount of dosh spent

MACHINE LEARNING TASK - Predict Transported (Binary Classification)

**To Do**
- Fill in missing values in a bunch of the columns
- Balance out the dataset if imbalanced
- Drop high cardinality columns


In [1]:
"""import tensorflow as tf #GPU'nun aktif olup olmadığını kontrol ettim.
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
"""

"import tensorflow as tf #GPU'nun aktif olup olmadığını kontrol ettim.\ndevice_name = tf.test.gpu_device_name()\nif device_name != '/device:GPU:0':\n  raise SystemError('GPU device not found')\nprint('Found GPU at: {}'.format(device_name))\n"

In [2]:
#from google.colab import drive #Google drive bağlandım.
#drive.mount('/content/gdrive')

In [3]:
!pip install kaggle

In [4]:
from google.colab import files # Kaggle uygulamsına bağlandım.
uploadded = files.upload()
!mkdir -p ~/.kaggle/ && mv  kaggle.json ~/.kaggle/ && chmod 600 ~/kaggle/kaggle.json

Saving kaggle.json to kaggle.json
chmod: cannot access '/root/kaggle/kaggle.json': No such file or directory


In [5]:
!kaggle competitions download -c spaceship-titanic

spaceship-titanic.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
!unzip spaceship-titanic.zip

Archive:  spaceship-titanic.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: sample_submission.csv   
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               


# Data Understanding

I use Pandas Profiling to do data analysis. It is data anlaysis tool, which leads to do fast and easy data undersatnding. We can save the report as notebook, json or html file on pc. I will save it as json file, then convert it pdf to easy reading.

In [7]:
#!git clone https://github.com/ydataai/ydata-profiling.git

In [8]:
#import pandas as pd
#from ydata_profiling import ProfileReport

In [9]:
import pandas as pd

In [10]:
df = pd.read_csv('/content/train.csv')

In [11]:
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [12]:
df.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [13]:
#profile = ProfileReport(df, title='Spaceship Titanic')

In [14]:
# As a file
#profile.to_file("Titanic_Report.json")

In [15]:
df.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

In [16]:
df.nunique()

PassengerId     8693
HomePlanet         3
CryoSleep          2
Cabin           6560
Destination        3
Age               80
VIP                2
RoomService     1273
FoodCourt       1507
ShoppingMall    1115
Spa             1327
VRDeck          1306
Name            8473
Transported        2
dtype: int64

# Data Preperation

For data preperation some requirements could be like;
- Need to fill if there is missing or faulty values on dataset
- Balance out dataset if there is imbalance on dataset
- Drop the columns, which has high cardinality, and etc.

In [17]:
# Check if we have wholly null rows if so drop?
df[df.isna().any(axis=1)].head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
7,0006_02,Earth,True,G/0/S,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,NaN,Candra Jacostaffey,True
10,0008_02,Europa,True,B/1/P,TRAPPIST-1e,34.0,False,0.0,0.0,NaN,0.0,0.0,Altardr Flatic,True
15,0012_01,Earth,False,NaN,TRAPPIST-1e,31.0,False,32.0,0.0,876.0,0.0,0.0,Justie Pooles,False
16,0014_01,Mars,False,F/3/P,55 Cancri e,27.0,False,1286.0,122.0,NaN,0.0,0.0,Flats Eccle,False
23,0020_03,Earth,True,E/0/S,55 Cancri e,29.0,False,0.0,0.0,NaN,0.0,0.0,Mollen Mcfaddennon,False


In [18]:
def split_cabin(x):
  if len(str(x).split('/')) < 3:
    return ['Missing', 'Missing', "Missing"]
  else:
    return str(x).split('/')

In [19]:
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [20]:
# Create a preprocessing function to transform our dataset
def preprocessing(df):
  # Fill missing values in homeplanet with missing
  df['HomePlanet'].fillna('Missing', inplace=True)
  # Cryosleep - highly correlated - drop na rows
  df['CryoSleep'].fillna('Missing', inplace=True)
  # Cabin preprocessing - extract Deck and Side
  df['TempCabin'] = df['Cabin'].apply(lambda x: split_cabin(x))
  df['Deck'] = df['TempCabin'].apply(lambda x: x[0])
  df['Side'] = df['TempCabin'].apply(lambda x: x[2])
  df.drop(['TempCabin', 'Cabin'], axis=1, inplace=True)
  # Destination
  df['Destination'].fillna('Missing', inplace=True)
  # Age
  df['Age'].fillna(df['Age'].mean(), inplace=True)
  # VIP - drop na rows
  df['VIP'].fillna('Missing', inplace=True)
  # Monetary spending columns
  df['RoomService'].fillna(0, inplace=True)
  df['FoodCourt'].fillna(0, inplace=True)
  df['ShoppingMall'].fillna(0, inplace=True)
  df['Spa'].fillna(0, inplace=True)
  df['VRDeck'].fillna(0, inplace=True)
  # Drop name due to high cardinality
  df.drop('Name', axis=1, inplace=True)
  # Drop remaining rows
  #df.dropna(inplace=True)

In [21]:
abt = df.copy()

In [22]:
preprocessing(abt)

In [23]:
# 8693 unique records
abt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8693 non-null   object 
 2   CryoSleep     8693 non-null   object 
 3   Destination   8693 non-null   object 
 4   Age           8693 non-null   float64
 5   VIP           8693 non-null   object 
 6   RoomService   8693 non-null   float64
 7   FoodCourt     8693 non-null   float64
 8   ShoppingMall  8693 non-null   float64
 9   Spa           8693 non-null   float64
 10  VRDeck        8693 non-null   float64
 11  Transported   8693 non-null   bool   
 12  Deck          8693 non-null   object 
 13  Side          8693 non-null   object 
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [24]:
abt.isnull().sum()

PassengerId     0
HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
Deck            0
Side            0
dtype: int64

In [25]:
abt.head()

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Side
0,0001_01,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,B,P
1,0002_01,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,F,S
2,0003_01,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,A,S
3,0003_02,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,A,S
4,0004_01,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,F,S


# Modelling

- A deep learning model is used for that project.
- Feature and Target values - X, y
- One hot encode any categorical features
- Train, holdout split
- Train on a bunch of algos

In [26]:
# Create feature columns
# Drop identifier column
X = abt.drop(['Transported', 'PassengerId'], axis=1)
# One hot encode
X = pd.get_dummies(X)
# Create target columns
y = abt['Transported']

In [27]:
# Create training and testing partitions
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [28]:
pip install keras==2.12.0

y


In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np

In [30]:
# Define your neural network model
def create_model(learning_rate=0.01,neurons_per_layer=64,dropout=0.2,activation ='softplus'):
    model = Sequential()
    model.add(Dense(10, input_dim=32, activation= activation))
    model.add(Dropout(dropout))
    model.add(Dense(10, input_dim=32, activation=activation))
    model.add(Dropout(dropout))
    model.add(Dense(10, input_dim=32, activation=activation))
    model.add(Dense(3, activation=activation))
    model.add(Dense(neurons_per_layer, activation=activation))
    model.add(Dense(1, activation=activation))
    adam0 = Adam(lr = learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=adam0, metrics=['accuracy'])
    return model

In [31]:
train_set_size = int(.67 * len(X_train))
num_epochs = 100
batch_size = 32

In [32]:
# Create your model
model = KerasClassifier(build_fn=create_model, epochs=num_epochs,batch_size =batch_size , verbose=1)

# Define the grid of hyperparameters
param_grid = {
    'learning_rate': [0.001, 0.01, 0.1],
    'dropout': [0.1,0.2,0.5],
    'activation': ['softplus','relu','sigmoid']
}

<ipython-input-32-9c0530fda923>:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, epochs=num_epochs,batch_size =batch_size , verbose=1)


In [33]:
# Perform Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train, y_train)

# Evaluate Results
print("Best Score:", grid_result.best_score_)
print("Best Parameters:", grid_result.best_params_)


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


127/127 [==============================] - 2s 3ms/step - loss: 0.9943 - accuracy: 0.6161
Epoch 2/100
127/127 [==============================] - 0s 4ms/step - loss: 0.6781 - accuracy: 0.7177
Epoch 3/100
127/127 [==============================] - 0s 3ms/step - loss: 0.7300 - accuracy: 0.6839
Epoch 4/100
127/127 [==============================] - 0s 3ms/step - loss: 0.6131 - accuracy: 0.7165
Epoch 5/100
127/127 [==============================] - 0s 3ms/step - loss: 0.6245 - accuracy: 0.7315
Epoch 6/100
127/127 [==============================] - 0s 3ms/step - loss: 0.6122 - accuracy: 0.7300
Epoch 7/100
127/127 [==============================] - 0s 3ms/step - loss: 0.6132 - accuracy: 0.7286
Epoch 8/100
127/127 [==============================] - 0s 3ms/step - loss: 0.7754 - accuracy: 0.5439
Epoch 9/100
127/127 [==============================] - 0s 3ms/step - loss: 0.6208 - accuracy: 0.6790
Epoch 10/100
127/127 [==============================] - 0s 2ms/step - loss: 0.5459 - accuracy: 0.7253
E

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


127/127 [==============================] - 1s 2ms/step - loss: 1.5021 - accuracy: 0.6359
Epoch 2/100
127/127 [==============================] - 0s 2ms/step - loss: 1.1228 - accuracy: 0.7136
Epoch 3/100
127/127 [==============================] - 0s 2ms/step - loss: 0.8323 - accuracy: 0.6032
Epoch 4/100
127/127 [==============================] - 0s 3ms/step - loss: 0.7946 - accuracy: 0.5265
Epoch 5/100
127/127 [==============================] - 0s 3ms/step - loss: 0.6983 - accuracy: 0.7032
Epoch 6/100
127/127 [==============================] - 0s 3ms/step - loss: 0.6398 - accuracy: 0.7281
Epoch 7/100
127/127 [==============================] - 0s 3ms/step - loss: 0.6302 - accuracy: 0.7276
Epoch 8/100
127/127 [==============================] - 0s 2ms/step - loss: 0.6154 - accuracy: 0.7294
Epoch 9/100
127/127 [==============================] - 0s 3ms/step - loss: 0.5924 - accuracy: 0.7321
Epoch 10/100
127/127 [==============================] - 0s 3ms/step - loss: 0.5765 - accuracy: 0.7326
E

...

127/127 [==============================] - 2s 4ms/step - loss: 0.7200 - accuracy: 0.5169
Epoch 2/100
127/127 [==============================] - 1s 4ms/step - loss: 0.6246 - accuracy: 0.6638
Epoch 3/100
127/127 [==============================] - 1s 4ms/step - loss: 0.5879 - accuracy: 0.7205
Epoch 4/100
127/127 [==============================] - 0s 3ms/step - loss: 0.5948 - accuracy: 0.7109
Epoch 5/100
127/127 [==============================] - 0s 3ms/step - loss: 0.5932 - accuracy: 0.7069
Epoch 6/100
127/127 [==============================] - 0s 3ms/step - loss: 0.5823 - accuracy: 0.7266
Epoch 7/100
127/127 [==============================] - 0s 3ms/step - loss: 0.5806 - accuracy: 0.7259
Epoch 8/100
127/127 [==============================] - 0s 3ms/step - loss: 0.5779 - accuracy: 0.7264
Epoch 9/100
127/127 [==============================] - 0s 3ms/step - loss: 0.5820 - accuracy: 0.7232
Epoch 10/100
127/127 [==============================] - 0s 3ms/step - loss: 0.5795 - accuracy: 0.7242
E

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


127/127 [==============================] - 2s 4ms/step - loss: 0.7338 - accuracy: 0.4979
Epoch 2/100
127/127 [==============================] - 1s 4ms/step - loss: 0.6543 - accuracy: 0.6175
Epoch 3/100
127/127 [==============================] - 1s 4ms/step - loss: 0.6352 - accuracy: 0.6685
Epoch 4/100
127/127 [==============================] - 1s 4ms/step - loss: 0.6120 - accuracy: 0.6951
Epoch 5/100
127/127 [==============================] - 1s 4ms/step - loss: 0.5938 - accuracy: 0.7123
Epoch 6/100
127/127 [==============================] - 1s 4ms/step - loss: 0.5903 - accuracy: 0.7096
Epoch 7/100
127/127 [==============================] - 0s 3ms/step - loss: 0.6095 - accuracy: 0.7035
Epoch 8/100
127/127 [==============================] - 0s 3ms/step - loss: 0.6056 - accuracy: 0.7054
Epoch 9/100
127/127 [==============================] - 0s 3ms/step - loss: 0.5992 - accuracy: 0.7111
Epoch 10/100
127/127 [==============================] - 0s 3ms/step - loss: 0.5938 - accuracy: 0.7141
E

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


191/191 [==============================] - 2s 8ms/step - loss: 0.6973 - accuracy: 0.4925
Epoch 2/100
191/191 [==============================] - 1s 3ms/step - loss: 0.6936 - accuracy: 0.5044
Epoch 3/100
191/191 [==============================] - 1s 3ms/step - loss: 0.6629 - accuracy: 0.6728
Epoch 4/100
191/191 [==============================] - 1s 3ms/step - loss: 0.5582 - accuracy: 0.7776
Epoch 5/100
191/191 [==============================] - 1s 3ms/step - loss: 0.5169 - accuracy: 0.7770
Epoch 6/100
191/191 [==============================] - 1s 3ms/step - loss: 0.5110 - accuracy: 0.7837
Epoch 7/100
191/191 [==============================] - 1s 3ms/step - loss: 0.5041 - accuracy: 0.7877
Epoch 8/100
191/191 [==============================] - 1s 3ms/step - loss: 0.5030 - accuracy: 0.7888
Epoch 9/100
191/191 [==============================] - 1s 3ms/step - loss: 0.5026 - accuracy: 0.7864
Epoch 10/100
191/191 [==============================] - 1s 3ms/step - loss: 0.5050 - accuracy: 0.7847
E

Saving the best model

---

In [34]:
best_model = grid_result.best_estimator_.model
best_model.save_weights('best_model_weights.h5')

In [35]:
#history_model = model.fit(X_train[:train_set_size], y_train[:train_set_size], epochs=num_epochs, batch_size=batch_size, verbose=0)

# Evuluation

In [36]:
score = best_model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.43013525009155273
Test accuracy: 0.796012282371521


In [37]:
# Read in the Test CSV Dataset
test_df = pd.read_csv('test.csv')
# Deep copy
abt_test = test_df.copy()
# Run through the preocessing pipeline
preprocessing(abt_test)
# One hot encode categorical variables
abt_test = pd.get_dummies(abt_test.drop('PassengerId', axis=1))

In [38]:
len(abt_test.columns)

32

In [39]:
len(X.columns)

32

In [42]:
yhat_test = best_model.predict(abt_test)

134/134 [==============================] - 0s 2ms/step


In [50]:
yhatdf = pd.DataFrame(yhat_test)
passdf = test_df['PassengerId']

In [71]:
submission1 = pd.concat([passdf,yhatdf],axis = 1)

In [75]:
submissionx = submission1.rename(index={'0': 'Transported'})

In [76]:
submissionx.tail()

,PassengerId,0
4272,9266_02,0.672276
4273,9269_01,0.700036
4274,9271_01,0.988545
4275,9273_01,0.700036
4276,9277_01,0.682210


# Deployment

In [78]:
submission1.to_csv('kaggle_submission.csv', index=False)

In [79]:
!kaggle competitions submit -c spaceship-titanic -m "initial gb model" -f "kaggle_submission.csv"

100% 78.1k/78.1k [00:00<00:00, 107kB/s]
Successfully submitted to Spaceship Titanic